In [9]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy import location
import folium

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
gets = requests.get(url)
soup = BeautifulSoup(gets.content, 'html.parser')
table = soup.find('table')
t_r = table.find_all('tr')
rows = []
for tr in t_r:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':  #Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if neighborhood == 'Not assigned':  #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough 
            neighborhood = borough   
        lst.append([postalcode, borough, neighborhood])

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)

df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()


Reading CSV file from link "https://cocl.us/Geospatial_data" to get the latitude and longitude of each neighborhood and add them to our data frame

In [6]:
df2= pd.read_csv('Geospatial_Coordinates.csv') # Reading csv
df['Latitude'] = np.nan
df['Longitude'] = np.nan
for i in df.index:
    ci = df2['Postal Code'] == df.loc[i, 'PostalCode']
    df.at[i, 'Latitude'] = df2.loc[ci, 'Latitude'].values
    df.at[i, 'Longitude'] = df2.loc[ci, 'Longitude'].values

In [ ]:
geolocator = Nominatim(user_agent="coursera")
add = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(add, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(add, index))

map_ = folium.Map(location=[latitude, longitude], zoom_start=12)

#adding markers to map usin labels
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3586cc',
        fill_opacity=0.5).add_to(map_)  
    
map_